# Test model on testing dataset

In the following script, we will test the saved model on a test dataset

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
from DataModel import DataModel, TensorDataSet, TensorTabDataSet
from sklearn import metrics
from torch.utils.data import DataLoader
from datetime import datetime
import torch
import torch.nn as nn
import pandas as pd

from tab_transformer_pytorch import TabTransformer, FTTransformer
from torch.utils.tensorboard import SummaryWriter



### Test FT-Transformer 

In [ ]:
data_encoding_type = 'label'
cols_to_ohe = ['priority', 'deck_on_vessel', 'is_reefer', 'is_hazardous', 'unitype_id']
dm = DataModel(encoding_type=data_encoding_type, cols_to_ohe=cols_to_ohe)
df = dm.get_df()
X,y = dm.get_inputs_targets()

def get_cols_unique_num():
    uniques = []
    for ind, i in enumerate(X.columns):
        if ind == 0 or ind == 1: continue
        # print(len(X[i].unique()))
        uniques.append(len(X[i].unique()))
    return tuple(uniques)
print(get_cols_unique_num())
unique_cat_num_tuple = get_cols_unique_num()
unique_cat_num_tuple = get_cols_unique_num()

model = FTTransformer(
    categories = unique_cat_num_tuple,
    num_continuous = 2,          
    dim = 64,                     
    dim_out = 1,                  
    depth = 6,                    
    heads = 8,                    
    attn_dropout = 0.2,           
    ff_dropout = 0.1              
)
device = torch.device('cpu')  # use cuda or cpu
model.to(device)

### Load saved model
replace the path with saved model path

In [ ]:
model.load_state_dict(torch.load("/Users/abdulrahman/Documents/DTU/deep-learning/02456_Deep_learning_2022_project_code/model_20230105_012058_0"))
model.eval()

In [ ]:

batch_size = 64
ts_tab_ds = TensorTabDataSet(data_type='test',normalize_num=False, encoding_type=data_encoding_type, cols_to_ohe=cols_to_ohe)

test_tab_loader =  DataLoader(ts_tab_ds, batch_size=batch_size)

test_results = pd.DataFrame({'pred':[], 'target':[]})

test_accuracy = []
pred_list=[]
targ_list=[]
for i ,data in enumerate(test_tab_loader):
    model.train(False)
    vinputs, vtargets = data
    vcat_inputs, vnum_inputs = vinputs[0], vinputs[1]
    vtargets = vtargets.type(torch.FloatTensor)
    vtargets = vtargets.unsqueeze(dim=1)
    pred = model(vcat_inputs, vnum_inputs)
    pred_list.extend(pred.cpu().detach().numpy().squeeze())
    targ_list.extend(vtargets.cpu().detach().numpy().squeeze())
    test_accuracy.append(metrics.r2_score(vtargets.cpu().detach().numpy().squeeze(), pred.cpu().detach().numpy().squeeze()))
plt.plot(test_accuracy)


### Test FNN

In [ ]:

class FFModel(nn.Module):
    def __init__(self, input_size):
        super(FFModel, self).__init__()  
        self.input_size =input_size
        self.l1=nn.Linear(self.input_size, self.input_size*2)
        self.l2=nn.Linear(self.input_size*2, self.input_size*10)
        self.l3=nn.Linear(self.input_size*10, self.input_size*20)
        self.l4=nn.Linear(self.input_size*20, self.input_size*50)
        self.l5=nn.Linear(self.input_size*50, self.input_size*55)
        self.l6=nn.Linear(self.input_size*55, 1)
        self.relu=nn.ReLU()
        self.dropout = nn.Dropout(0.30)
    def forward(self, x):
        out=self.l1(x)
        out=self.relu(out)
        self.dropout(out)
        out=self.l2(out)
        out=self.relu(out)
        self.dropout(out)
        out=self.l3(out)
        out=self.relu(out)
        self.dropout(out)
        out=self.l4(out)
        out=self.relu(out)
        self.dropout(out)
        out=self.l5(out)
        out=self.relu(out)
        out=self.l6(out)
        return out
# encode_type = ''
# cols_to_label_encode=['place_on_board']
data_encoding_type = 'label'
cols_to_ohe = ['priority', 'deck_on_vessel', 'is_reefer', 'is_hazardous', 'unitype_id']
dm = DataModel(encoding_type=data_encoding_type, normalize_num=False, cols_to_ohe=cols_to_ohe)
df = dm.get_df()
input_size = dm.get_inputs_targets()[0].shape[1]
model = FFModel(input_size)

### Load saved model
replace the path with saved model path

In [ ]:
model.load_state_dict(torch.load("PATH"))
model.eval()

In [ ]:

batch_size = 64
ts_ff_ds = TensorDataSet(data_type='test',normalize_num=False, encoding_type=data_encoding_type, cols_to_ohe=cols_to_ohe)

test_ff_loader =  DataLoader(ts_ff_ds, batch_size=batch_size, shuffle=True)

In [ ]:
test_ff_results = pd.DataFrame({'pred':[], 'target':[]})

test_ff_accuracy = []
pred_ff_list=[]
targ_ff_list=[]
for i ,data in enumerate(test_ff_loader):
    model.train(False)
    vinputs, vtargets = data
    # vcat_inputs, vnum_inputs = vinputs[0], vinputs[1]
    vtargets = vtargets.type(torch.FloatTensor)
    vtargets = vtargets.unsqueeze(dim=1)
    pred = model(vinputs)
    pred_ff_list.extend(pred.cpu().detach().numpy().squeeze())
    targ_ff_list.extend(vtargets.cpu().detach().numpy().squeeze())
    # print("vtargets.cpu().detach().numpy().squeeze()")

    # if i == 0:
    #     break
    test_ff_accuracy.append(metrics.r2_score(vtargets.cpu().detach().numpy().squeeze(), pred.cpu().detach().numpy().squeeze()))
plt.plot(test_ff_accuracy)
